# NLP Visualizations

In [2]:
!aws s3 ls


2023-08-29 23:43:16 sagemaker-studio-692960231031-wo7kgoszj2g
2023-08-29 23:50:01 sagemaker-us-east-1-692960231031
2023-08-30 00:34:21 vad49
2023-09-16 16:02:10 vad49-labdata


In [19]:
!aws s3 ls s3://project17-bucket-alex/stories-and-books-nlp/


2023-11-18 15:01:01          0 _SUCCESS
2023-11-18 15:00:57    2215290 part-00000-bd42c6b5-9806-4914-961a-007650c34d62-c000.snappy.parquet
2023-11-18 15:00:58    2058430 part-00001-bd42c6b5-9806-4914-961a-007650c34d62-c000.snappy.parquet
2023-11-18 15:00:59      20541 part-00035-bd42c6b5-9806-4914-961a-007650c34d62-c000.snappy.parquet
2023-11-18 15:00:59       9836 part-00051-bd42c6b5-9806-4914-961a-007650c34d62-c000.snappy.parquet
2023-11-18 15:00:59      22532 part-00053-bd42c6b5-9806-4914-961a-007650c34d62-c000.snappy.parquet
2023-11-18 15:01:00      25358 part-00059-bd42c6b5-9806-4914-961a-007650c34d62-c000.snappy.parquet


In [ ]:
if True is True: # set to true only for the first un
    # Setup - Run only once per Kernel App
    %conda install openjdk -y

    # install PySpark
    %pip install pyspark==3.2.0 s3fs pyarrow

    # restart kernel
    from IPython.core.display import HTML
    HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import col, length, isnan, when, count, regexp_extract, weekofyear, hour, avg, to_date, unix_timestamp, lit, corr


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 150) 
#pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)


In [2]:
# Import pyspark and build Spark session

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    #.config("spark-jars-packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3")\
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6c101a14-40de-44a7-ae1b-09c0190d2689;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 383ms :: artifacts dl 30ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

3.4.0


In [9]:
# read in the data

all_model_text = spark.read.parquet("s3a://project17-bucket-alex/stories-and-books-nlp/")

display(f"shape: ({all_model_text.count()}, {len(all_model_text.columns)})")
all_model_text.show()

'shape: (7830, 3)'

+--------------------+--------------------+--------------------+
|                text|       custom_tokens|         text_as_int|
+--------------------+--------------------+--------------------+
|       Metamorphosis|     [metamorphosis]|[29, 21, 36, 17, ...|
|      by Franz Kafka|[by,  , franz,  ,...|[18, 41, 1, 22, 3...|
|Translated by Dav...|[translated,  , b...|[36, 34, 17, 30, ...|
|                   I|                 [i]|                [25]|
|One morning, when...|[one,  , morning,...|[31, 30, 21, 1, 2...|
|“What’s happened ...|[whats,  , happen...|[39, 24, 17, 36, ...|
|Gregor then turne...|[gregor,  , then,...|[23, 34, 21, 23, ...|
|“Oh, God”, he tho...|[oh, ,,  , god, ,...|[31, 24, 3, 1, 23...|
|He slid back into...|[he,  , slid,  , ...|[24, 21, 1, 35, 2...|
|And he looked ove...|[and,  , he,  , l...|[17, 30, 20, 1, 2...|
|He was still hurr...|[he,  , was,  , s...|[24, 21, 1, 39, 1...|
|The first thing h...|[the,  , first,  ...|[36, 24, 21, 1, 2...|
|It was a simple m...|[it